# Milvus Agent

### Milvus

In [45]:
import os
from pymilvus import MilvusClient, connections, FieldSchema, CollectionSchema, DataType, Collection, utility

# Replace uri and token with your own
client = MilvusClient(
    uri=os.getenv("MILVUS_ENDPOINT"),
    token=os.getenv("MILVUS_API_KEY") # API key or a colon-separated cluster username and password
)

# Connect to cluster
connections.connect(
  alias='default',
  # Public endpoint obtained from Zilliz Cloud
  uri=os.getenv("MILVUS_ENDPOINT"),
  # API key or a colon-separated cluster username and password
  token=os.getenv("MILVUS_API_KEY")
)

In [46]:
from pymilvus import Collection
collection = Collection("News_2023")      # Get an existing collection.
collection.load()

In [47]:
from typing import List, Any
from langchain.tools import tool
from langchain.pydantic_v1 import BaseModel, Field
from sentence_transformers import SentenceTransformer
# Cargar el modelo de Sentence Transformers y moverlo a CUDA si está disponible
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)

@tool('milvus_query_results')
def milvus_query_results(query: dict) -> List[Any]:
    "Gives a list of document found by the query"
    query = query['title']
    vect = model.encode(query)
    results = collection.search(
        data=[vect], 
        anns_field="vector", 
        param={"metric_type": "L2", "params": {"nprobe": 1}}, 
        limit=5, 
        expr=None,
        consistency_level="Strong"
    )
    ids = results[0].ids
    res = collection.query(
        expr =f"""Auto_id in {ids}""" , 
        output_fields = ["body"],
        consistency_level="Strong"
    )
    return res

# debug


In [18]:
a = {"rank":100,
"id":"448075995546733363",
"score":"6.5996346",
"dataType":"news",
"dateTimePub":"2023-05-02 19:22:00+00:00",
"Auto_id":448075995546733400,
"vector":[-0.09536351,-0.018581286,-0.017350093,-0.12269653,0.18249905,0.13691229,-0.051429454,0.04772178,0.13194075,0.07268002,0.11690043,0.30491424,0.07573706,0.048195593,0.07615636,0.015191397,-0.024883967,-0.06935531,0.027576813,0.12269305,0.00661877,-0.16038519,0.14568141,0.035197053,0.07685859,0.015172271,-0.13197769,-0.021624755,-0.18772297,-0.17756143,-0.07141833,0.023578372,0.027295128,-0.013786257,0.10980527,-0.053779535,0.07168948,-0.0013504851,-0.24489671,-0.119475946,-0.09151496,-0.13940004,0.019152213,-0.093431875,-0.0154781155,0.016518693,-0.012374892,-0.05537989,-0.2661745,0.028235897,0.07142044,-0.05416686,-0.0390324,-0.031918462,-0.031115584,-0.07178099,0.023655102,0.10683921,-0.07076112,-0.01649535,-0.07847581,-0.11521027,-0.05321051,0.07611768,0.05987252,-0.06459176,0.043336205,-0.103966735,0.06354885,0.03930246,-0.032041114,-0.043630324,-0.02802879,-0.0155294165,0.11891474,0.08610359,-0.010282127,0.13050073,0.1508202,-0.22096854,0.11752796,-0.024599921,0.03131011,-0.027514629,-0.05080875,0.045154557,-0.043037258,0.017501594,0.021306446,-0.16782264,-0.17585647,0.01401872,0.27563173,-0.009840605,0.07395008,-0.018436212,0.008207006,0.08554071,0.055805713,0.1288198,-0.020472847,-0.03625696,-0.18749686,-0.11663776,-0.008424758,-0.1405205,0.18850288,-0.161086,-0.018813998,-0.0019631195,-0.08425469,-0.030923769,0.077334955,-0.086956084,0.096051335,-0.015178316,-0.06325907,0.1274518,-0.039295416,0.027459014,-0.11114648,-0.12871248,0.05399959,-0.034501508,0.02652131,0.019759182,0.14360479,0.026895102,0.029065356,-0.120457076,-0.06761839,-0.14628512,-0.05672548,-0.010576123,0.075598374,0.02961963,0.06799347,-0.09534201,0.025691811,0.067902714,0.03770698,0.3016634,0.05055288,0.03583135,0.023918046,0.1672086,0.09134467,-0.06589576,0.04832609,0.034699377,0.14615819,0.024327472,0.08977108,0.06397565,-0.09879513,0.065217204,0.06108924,0.069831654,-0.011885773,0.052261412,-0.045385424,0.0035886983,0.12642115,0.09225933,0.114664346,-0.060498822,-0.09319426,-0.09206743,0.04895942,0.24697724,-0.19084415,-0.23708403,0.079770535,-0.05779876,-0.04836727,-0.12328758,0.051453587,0.114496045,-0.049792346,0.008941798,-0.039882652,-0.089992374,0.11606027,0.0148419365,-0.07177505,-0.20030549,0.10728097,0.004206855,-0.041069105,0.08076356,-0.17876273,0.011166595,0.12235014,0.3415074,0.059670016,0.03319464,-0.052310742,0.077035755,0.05859136,-0.057622194,-0.36455667,0.014672197,-0.13280462,-0.06829305,-0.0894337,-0.03174439,0.133008,-0.15781277,0.15638554,0.037571557,0.024511661,-0.13863882,-0.05692687,-0.10303365,0.11133145,-0.07365389,-0.21101911,0.16192567,-0.082310416,0.11762484,-0.08802287,-0.036474176,0.06278604,-0.21600604,-0.22477931,-0.08927704,-0.19232397,-0.004092955,-0.1532667,0.06389594,-0.07305354,-0.066961184,0.07452362,0.15642147,-0.01222572,-0.13945884,-0.001326777,0.056221187,-0.12199307,-0.20669577,-0.14937165,-0.10188657,0.017001614,0.037054915,0.12931412,-0.08464402,-0.080282256,0.07025835,-0.011251997,0.09582301,0.06554506,-0.17041276,0.008375335,-0.032498963,0.09974381,0.048293166,-0.085887656,0.08465653,0.1591787,0.04964045,0.11402033,0.11072142,0.02235236,-0.037369475,0.1725519,0.1029965,-0.19827843,0.047394,-0.02930706,-0.044773806,-0.31304824,-0.03713318,0.07225484,-0.14366387,0.042981125,-0.20980954,0.020333884,-0.18438488,-0.052506313,0.16613,0.041627727,0.059528355,0.1630367,0.028674658,0.046024613,-0.16098809,-0.096971795,-0.2680946,0.06187685,0.071498066,0.09577328,0.05031075,0.08304477,0.06533778,0.16243584,-0.0933595,0.042339906,-0.14694858,-0.085878566,0.103488915,-0.05886639,0.069146514,-0.0004322431,-0.04040499,0.124432474,-0.054998875,0.058639817,0.057270005,0.09368582,-0.03369966,-0.0030608857,-0.16608459,-0.11861855,0.124764554,0.09695947,0.01049341,-0.028847292,0.13263707,0.027882468,-0.25028598,0.07719878,0.038105235,0.055952743,0.07404498,0.119886644,0.09328315,0.03673497,-0.0019275813,0.03597531,-0.030587241,0.10162406,0.11553602,0.019756004,-0.010583381,0.021171816,-0.03566678,-0.040099293,0.016961316,-0.10072513,0.011676127,0.23124361,0.18499005,-0.006994809,-0.14544484,0.011323348,0.016413819,0.17280805,0.16296268,-0.03782331,-0.06617904,-0.09126413,0.005789415,-0.09799647,0.01485052,-0.11786923,0.019148167,0.01677419,-0.030063288,0.03182601,-0.08226896,0.04402654,-0.06418039,-0.005053025,0.16724017,0.13339409,0.058648616,-0.10214265,0.030780653,-0.08792026,-0.074504025,-0.0908303,-0.015362122,0.05891101,0.10935603,0.19564383,-0.120186985,0.09486053,0.033717208,-0.07977885,0.07083815,0.07101032,0.07413533,0.118418775,0.008283526],
"body":"máximo histórico de US$494 millones en 2021. La AMPTP también rechazó la demanda del gremio de que se prohibiera el uso de bots de IA para escribir o reescribir material, y en su lugar ofreció sostener \"reuniones anuales para discutir los avances en tecnología\". ¿Qué pasó durante la última huelga de guionistas? Muchos lectores pueden recordar cómo cambiaron las producciones de televisión y cine durante la etapa de los confinamientos por la pandemia, pero es posible que algunos no recuerden los efectos similares de la última gran hue","title":"La huelga de guionistas que paraliza Hollywood y qué puede retrasar el estreno de tu serie o programa favoritos","uri":"7525612905","url":"https://www.24horas.cl/internacional/noticias-bbc/la-huelga-de-guionistas-que-paraliza-hollywood-y-que-puede-retrasar-el"}

In [31]:
results = collection.search(
    data = [a['vector']],
    anns_field = "vector",
    param = {"mertic_type":"L2", "params":{"nprobe":1}},
    limit = 6,
    expr = None,
    consistency_level = "Strong"
)
a = results[0][0].id

In [32]:
a

448075995546733363

In [35]:
b = "Auto_id in [448075995546733400]"

In [41]:
res = collection.query(
    expr =b, 
    output_fields = ["body"],
    consistency_level="Strong"
)
res

[{'body': 'transportó a unos 2.000 pasajeros, incluso incluyó a iraníes. Los archirrivales gobiernos de Riad y Teherán se movieron recientemente hacia un acercamiento cauteloso, incluida la reapertura de sus embajadas y consulados. "Es nuestra suerte. Esperamos que haya paz entre nuestros países", comentó Nazli, una ingeniera civil de 32 años, mientras desembarcaba en Yeda con su esposo ingeniero, quien también ha trabajado durante años como ingeniero en Sudán.',
  'Auto_id': 448075995546733400}]

In [39]:
vect = model.encode('Maximiliano Falcón destaca su presente en Colo Colo:')
results = collection.search(
    data=[vect], 
    anns_field="vector", 
    param={"metric_type": "L2", "params": {"nprobe": 1}}, 
    limit=5, 
    expr=None,
    consistency_level="Strong"
)

In [42]:
results[0].ids

[448075995546508619,
 448075995546588402,
 448075995546691603,
 448075995546856347,
 448075995546688124]

In [44]:
ids = results[0].ids
res = collection.query(
    expr =f"""Auto_id in {ids}""" , 
    output_fields = ["body"],
    consistency_level="Strong"
)
res

[{'body': 'Maximiliano Falcón destaca su presente en Colo Colo: \'He madurado mucho, estoy orgulloso de mí mismo\' El defensa de Colo Colo, Maximiliano Falcón, lamentó el estar suspendido de cara al encuentro de este sábado, donde recibirán a O\'Higgins por el Campeonato Nacional 2023 antes de viajar a Belo Horizonte para cerrar la llave por Copa Sudamericana ante América Mineiro. "El equipo está bien, al que le toque jugar lo hará bien. Este triunfo sirvió para subir el ánimo del equipo. Tenemos que ir allá, no será fácil, pero daremos lo',
  'Auto_id': 448075995546508619},
 {'body': '"Si la oferta es buena, bienvenido sea": Falcón no le cierra la puerta a un posible adiós de Colo Colo ¿Comienza a verse lejos de Macul? Esa es, precisamente, la interrogante que buscan resolver en Colo Colo respceto al futuro de Maximiliano Falcón en el equipo. Y es que antes de viajar hacia Venezuela, de cara al duelo que enfrentarán los albos frente a Monagas por el grupo F de la Copa Libertadores, el

# LLM and Agent-Chain

In [48]:
tools = [milvus_query_results]

In [49]:
from langchain_together import Together
import os
from dotenv import load_dotenv
load_dotenv()

llm = Together(
    model="mistralai/Mixtral-8x7B-Instruct-v0.1",
    temperature=0,
    max_tokens=1024,
    top_k=1,
    together_api_key=os.getenv("TOGETHER_API_KEY")
)

In [19]:

DEFAULT_TEMPLATE = """Eres un asistente de IA especializado en verificar hechos sobre la realidad chilena en el año 2023. 
Tu deber es verificar la veracidad de las afirmaciones presentadas relacionadas con eventos, situaciones o datos sobre Chile durante el año 2023, y responder si son verdaderas o falsas de manera concisa, sin proporcionar explicaciones adicionales. 
Para revisar si la afirmacion es correcta, harás una consulta a una base de datos con la siguiente herramienta:

Herramientas:
{tools}

Si no tienes suficiente información en tu base de conocimientos para determinar la veracidad de una afirmación sobre la realidad chilena en 2023, debes responder honestamente que no lo sabes. 
Tus respuestas deben ser siempre en español.

Luego de obtener las informacion de la base de datos, debes responder unicamente con 'Verdadero', 'Falso' o 'No lo sé' a la afirmación sobre la realidad chilena en 2023: 
    
Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
    "action": $TOOL_NAME,
    "action_input": $INPUT
}}```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
    Observation: action result
    ... (repeat Thought/Action/Observation N times)
    Thought: I know what to respond
        Action:
        ```
            {{
                "action": "Final Answer",
                "action_input": "Final response to human"
            }}
            
For example, if you want to use a tool to make a query to the vectore store database, your $JSON_BLOB might look like this:

```
{{
    'action': 'vector_store_query_json', 
    'action_input': {{'query': 'Example query'}}
}}```

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation'
"""

In [50]:
DEFAULT_TEMPLATE = """You are a helpfull AI assistant that helps to makes querys to a vectore store database given a human consult of a specific topic.
The AI gives information and explanation from the results of the query.
The AI is talkative and provides a lot of specific details of its context.
If the AI does not know the answer to the question, it will truthfully sai it does not know.

You have access to the following tools:
{tools}

Use a json blob to specify a tool by providing an action key (tool name) and an action_input key (tool input).

Valid "action" values: "Final Answer" or {tool_names}

Provide only ONE action per $JSON_BLOB, as shown:

```
{{
    "action": $TOOL_NAME,
    "action_input": $INPUT
}}```

Follow this format:

Question: input question to answer
Thought: consider previous and subsequent steps
Action:
```
$JSON_BLOB
```
    Observation: action result
    ... (repeat Thought/Action/Observation N times)
    Thought: I know what to respond
        Action:
        ```
            {{
                "action": "Final Answer",
                "action_input": "Final response to human"
            }}
            
For example, if you want to use a tool to make a query to the vectore store database, your $JSON_BLOB might look like this:

```
{{
    'action': 'vector_store_query_json', 
    'action_input': {{'query': 'Example query'}}
}}```

Begin! Reminder to ALWAYS respond with a valid json blob of a single action. Use tools if necessary. Respond directly if appropriate. Format is Action:```$JSON_BLOB```then Observation'
"""


In [51]:
from langchain import hub
prompt = hub.pull("hwchase17/structured-chat-agent")
#prompt[0].prompt.template = prompt[0].prompt.template[0:394] + "```\n\n\nFor example, if you want to use a tool to get an image's caption, your $JSON_BLOB might look like this:\n\n```\n{{\n    'action': 'image_captioner_json', \n    'action_input': {{'query': 'images url'}}\n}}```" + "```\n\n\nIf you want to use a tool to search for information, your $JSON_BLOB might look like this:\n\n```\n{{\n    'action': 'tavily_search_results_json', \n    'action_input': {{'query': 'Example search query'}}\n}}```"  +prompt[0].prompt.template[394:]


In [52]:
prompt[0].prompt.template = DEFAULT_TEMPLATE

In [53]:
from langchain.agents import create_openai_functions_agent
from langchain.agents import AgentExecutor, create_openai_tools_agent, create_structured_chat_agent
#agent = create_openai_tools_agent(llm,tools,prompt)
agent = create_structured_chat_agent(llm, tools, prompt)

In [54]:
from langchain.agents import AgentExecutor

agent_executor = AgentExecutor(agent=agent, tools=tools, verbose=True, handle_parsing_errors=True)

In [55]:
agent_executor.invoke({"input": "El ministro de Justicia Luis Cordero asume su cargo en enero del 2023"})



> Entering new AgentExecutor chain...


Action:
```
{
    "action": "milvus_query_results",
    "action_input": {
        "query": {
            "title": "Ministro de Justicia Luis Cordero",
            "type": "object"
        }
    }
}
```
[{'body': 'Luis Cordero: "La potestad de indultar en ningún caso es intromisión en otro poder del Estado" El abogado Luis Cordero asumió formalmente como ministro de Justicia, luego de la renuncia de Marcela Ríos en medio de la polémica por los indultos presidenciales. El Presidente Gabriel Boric dijo que "es necesario avanzar con medidas concretas que garanticen el acceso equitativo a la justicia, ampliando el ejercicio efectivo de derechos, igualdad de género y también siempre pensando en la protección de las víctimas". Posterior a la reunión con el Mandatario, el nuevo secretario de Estado se refir', 'Auto_id': 448075995546843085}, {'body': 'Luis Cordero asume como ministro de Justicia y evita opinar de indultos para ver "papelitos primero" En

{'input': 'El ministro de Justicia Luis Cordero asume su cargo en enero del 2023',
 'output': {'body': "Luis Cordero assumed the position of Minister of Justice in January 2023, following Marcela Ríos' resignation amidst controversy over presidential pardons. The new minister has been cautious in his statements regarding the pardons, stating that he needs to review the documents first before commenting. The appointment of Luis Cordero as Minister of Justice is generally viewed positively, with officials praising his legal expertise and ability to foster transversal agreements."}}

# Insertar Data y revisar si estaba bien

In [42]:
import pandas as pd
from sentence_transformers import SentenceTransformer
from langchain.text_splitter import SentenceTransformersTokenTextSplitter
from tqdm.auto import tqdm
import torch
import os
from pymilvus import MilvusClient, connections, FieldSchema, CollectionSchema, DataType, Collection, utility
import json
import re
import argparse
import os.path
from dotenv import load_dotenv

# Carga las variables de entorno desde el archivo .env
load_dotenv()

ruta_archivo = '../Data/2023_cleaned_embeddings.json'


with open(ruta_archivo, 'r') as f:
    data = json.load(f)

# Replace uri and token with your own
client = MilvusClient(
    uri=os.getenv("MILVUS_ENDPOINT"),
    token=os.getenv("MILVUS_API_KEY")  # API key or a colon-separated cluster username and password
)

# Connect to cluster
connections.connect(
    alias='default',
    # Public endpoint obtained from Zilliz Cloud
    uri=os.getenv("MILVUS_ENDPOINT"),
    # API key or a colon-separated cluster username and password
    token=os.getenv("MILVUS_API_KEY")
)
print(os.getenv("MILVUS_API_KEY"))
# Definimos el tamaño del bloque
block_size = 1000

# Iteramos sobre los bloques
for i in range(0, len(data['rows']), block_size):
    res = client.insert(
        collection_name='News_2023',
        data=data['rows'][i:i + block_size]
    )
    print('Insertando bloque ' + str(i))

print("Data insertada correctamente")

9be85f94cdb6c4b3ee58a9dffca1d3f9db182ee9cd46be3ea8ced4df882f7a30439a2f48f42ea1763e30fa5a2d5a4807a325ee65
Insertando bloque 0
Insertando bloque 1000
Insertando bloque 2000
Insertando bloque 3000
Insertando bloque 4000
Insertando bloque 5000
Insertando bloque 6000
Insertando bloque 7000
Insertando bloque 8000
Insertando bloque 9000
Insertando bloque 10000
Insertando bloque 11000
Insertando bloque 12000
Insertando bloque 13000
Insertando bloque 14000
Insertando bloque 15000
Insertando bloque 16000
Insertando bloque 17000
Insertando bloque 18000
Insertando bloque 19000
Insertando bloque 20000
Insertando bloque 21000
Insertando bloque 22000
Insertando bloque 23000
Insertando bloque 24000
Insertando bloque 25000
Insertando bloque 26000
Insertando bloque 27000
Insertando bloque 28000
Insertando bloque 29000
Insertando bloque 30000
Insertando bloque 31000
Insertando bloque 32000
Insertando bloque 33000
Insertando bloque 34000
Insertando bloque 35000
Insertando bloque 36000
Insertando bloque 37

In [10]:
import pandas as pd

In [11]:
df = pd.read_json('../Data/2023_cleaned_embeddings.json', encoding='utf-8')

In [12]:
df = df.sample(20)

In [15]:
new_df = pd.json_normalize(df['rows'])

In [16]:
new_df

,uri,dataType,title,dateTimePub,body,url,vector
0,7448959020,news,"""Informe para una academia"": Performance basad...",2023-03-15 21:03:00+00:00,"Claudio Marín. El director agrega que ""estos l...",https://www.biobiochile.cl/noticias/artes-y-cu...,"[0.200326278805732, 0.084796532988548, -0.0715..."
1,7334009958,news,Corte de Santiago confirma multa a TVN por inf...,2023-01-04 20:38:00+00:00,"en primer término, que el CNTV ha modificado e...",https://www.biobiochile.cl/noticias/artes-y-cu...,"[-0.019818533211946, -0.370678305625915, 0.139..."
2,7567837207,news,17 hechos violentos investiga la Fiscalía en L...,2023-05-29 13:06:00+00:00,"i, confirmó que hasta ahora hay 17 hechos que ...",https://www.biobiochile.cl/noticias/nacional/r...,"[0.012530170381069001, 0.011203557252883, -0.0..."
3,7564378412,news,Gonzalo Valenzuela interpone querella por amen...,2023-05-26 22:41:00+00:00,Gonzalo Valenzuela interpone querella por amen...,https://www.biobiochile.cl/noticias/espectacul...,"[-0.042528636753559, 0.06623015552759101, -0.2..."
4,7618969308,news,La Producción Industrial cayó en mayo y el Com...,2023-06-30 14:29:00+00:00,", principalmente, ""la disminución en otras act...",https://www.biobiochile.cl/noticias/economia/a...,"[0.086207218468189, 0.034578330814838, 0.12727..."
5,7399549759,news,Así evaluaron los hinchas de Colo Colo a Matía...,2023-02-14 02:09:00+00:00,Así evaluaron los hinchas de Colo Colo a Matía...,https://www.24horas.cl/deportes/futbol-naciona...,"[-0.013411339372396, 0.10699708759784601, -0.2..."
6,7443749111,news,Pablo Chill-E anuncia nuevas colaboraciones co...,2023-03-13 01:37:00+00:00,Pablo Chill-E anuncia nuevas colaboraciones co...,https://www.adnradio.cl/espectaculos/2023/03/1...,"[-0.10399141907691901, -0.285190522670745, 0.1..."
7,7568496034,news,Número de personas en listas de espera por cir...,2023-05-29 23:42:00+00:00,"ley para crear el Servicio de Salud Virtual, q...",https://www.adnradio.cl/nacional/2023/05/29/nu...,"[0.062103763222694, -0.051294051110744004, -0...."
8,7529322172,news,Van Klaveren confirma llegada de avión venezol...,2023-05-04 22:03:00+00:00,"de un enfoque y un abordaje regional, porque e...",https://www.chvnoticias.cl/nacional/canciller-...,"[-0.043996516615152005, -0.075283080339431, -0..."
9,7505504870,news,"ONU: A fines de año, la población de India ...",2023-04-19 20:42:00+00:00,un censo de población en la India -que se iba ...,https://cooperativa.cl/noticias/mundo/india/on...,"[-0.093199223279953, 0.014530253596603001, -0...."


# inutil


In [ ]:
from langchain.vectorstores import Milvus
from langchain.llms import OpenAI
from langchain.chains import RetrievalQA
import os

MILVUS_HOST = os.environ.get('MILVUS_HOST', 'localhost')
MILVUS_PORT = os.environ.get('MILVUS_PORT', 19530)

vector_store = Milvus(
    host=MILVUS_HOST,
    port=MILVUS_PORT,
    collection_name="tu_coleccion_milvus",
    embedding_function=OpenAI().embed_query,
    embedding_dim=1536,
)

qa = RetrievalQA(
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
)
query = "¿Cuál es la capital de Francia?"
result = qa.run(query)
print(result)

In [17]:
import os
from pymilvus import MilvusClient, connections
from sentence_transformers import SentenceTransformer
from langchain.vectorstores import Milvus
from langchain.chains import RetrievalQA
from dotenv import load_dotenv

# Carga las variables de entorno desde el archivo .env
load_dotenv()

# Carga el modelo y define la función de embedding
model_name = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
model = SentenceTransformer(model_name)

def embedding_func(text):
    return model.encode([text], convert_to_tensor=False)[0]

# Replace uri and token with your own
client = MilvusClient(
    uri=os.getenv("MILVUS_ENDPOINT"),
    token=os.getenv("MILVUS_API_KEY")  # API key or a colon-separated cluster username and password
)

# Connect to cluster
connections.connect(
    alias='default',  # Public endpoint obtained from Zilliz Cloud
    uri=os.getenv("MILVUS_ENDPOINT"),
    token=os.getenv("MILVUS_API_KEY")  # API key or a colon-separated cluster username and password
)

collection_name = "News_2023"
dimension = 384  # La dimensión del embedding para este modelo

# Configura el retriever de Milvus en LangChain
vector_store = Milvus(
    client=client,
    collection_name="News_2023",
    embedding_function=embedding_func,
    embedding_dim=384,
)

# Crea una instancia de RetrievalQA y utiliza el retriever de Milvus
qa = RetrievalQA(
    retriever=vector_store.as_retriever(),
    return_source_documents=True,
)

# Realiza una consulta
query = "El ministro de Justicia Luis Cordero asume su cargo en enero del 2023"
result = qa.run(query)
print(result)

TypeError: Milvus.__init__() got an unexpected keyword argument 'client'